In [1]:
import numpy as np
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/Colab/LAB2"
%ls

Mounted at /content/drive
/content/drive/MyDrive/Colab/LAB2
lab2_review.ipynb  test_dataset.npy  train_dataset.npy
lab2_task.ipynb    test_labels.npy   train_labels.npy


In [5]:
import numpy as np

test_dataset = np.load("test_dataset.npy")
test_labels = np.load("test_labels.npy")
train_dataset = np.load("train_dataset.npy")
train_labels = np.load("train_labels.npy")

print(train_labels.reshape(-1, 1).shape)
print(np.arange(3).shape)
deltas = (np.arange(3) == train_labels.reshape(-1, 1))
print(deltas) 


# for i in range(900):
#   for j in range(2642):
#     if (train_dataset[i][j] == True and train_labels[i] == 0):
#       yes_label[0][j] = yes_label[0][j] + 1
#     if (train_dataset[i][j] == True and train_labels[i] == 1):
#       yes_label[1][j] = yes_label[1][j] + 1
#     if (train_dataset[i][j] == True and train_labels[i] == 2):
#       yes_label[2][j] = yes_label[2][j] + 1

# training_set = np.append(train_dataset, np.reshape(train_labels, (900, 1)), axis = 1)
# yes_label = np.sum((training_set[:,0] == 1), where = (training_set[:, 2642] == 0))
# print(yes_label)

# print(yes_label.shape)
# print(yes_label)
# print(np.sum(yes_label, axis = 1))

(900, 1)
(3,)
[[ True False False]
 [False  True False]
 [ True False False]
 ...
 [ True False False]
 [False False  True]
 [False  True False]]


In [ ]:
import numpy as np
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/Colab/LAB2"

test_dataset = np.load("test_dataset.npy")
test_labels = np.load("test_labels.npy")
train_dataset = np.load("train_dataset.npy")
train_labels = np.load("train_labels.npy")

class NaiveBayesClassifier:
  def __init__(self):
    self.train_dataset = None
    self.train_labels = None
    self.train_size = 0
    self.num_features = 0
    self.num_classes = 0

  def fit(self, train_dataset, train_labels):
    self.train_dataset = train_dataset
    self.train_labels = train_labels
    # TODO
    self.train_size = train_dataset.shape[0]
    self.num_features = train_dataset.shape[1]
    self.num_classes = np.amax(train_labels) + 1
  
  def estimate_class_prior(self):
    # TODO
    positive_prior = (np.sum(self.train_labels == 0)+ 1)/(self.train_size + self.num_classes)
    neutral_prior  = (np.sum(self.train_labels == 1)+ 1)/(self.train_size + self.num_classes)
    negative_prior = (np.sum(self.train_labels == 2)+ 1)/(self.train_size + self.num_classes)
    
    class_prior = np.array([positive_prior, neutral_prior, negative_prior])

    return class_prior
    # return class_prior

  def estimate_likelihoods(self):
    # TODO
    
    # evidence = np.zeros((self.num_classes, self.num_features))
    
    # for i in range(900):
    #   for j in range(2642):
    #     if (self.train_dataset[i][j] == True and self.train_labels[i] == 0):
    #       evidence[0][j] = evidence[0][j] + 1
    #     if (self.train_dataset[i][j] == True and self.train_labels[i] == 1):
    #       evidence[1][j] = evidence[1][j] + 1
    #     if (self.train_dataset[i][j] == True and self.train_labels[i] == 2):
    #       evidence[2][j] = evidence[2][j] + 1

    # print(evidence.dtype)
    # evidence[0] = (evidence[0] + 1)/(np.sum(self.train_labels == 0) + 2)
    # evidence[1] = (evidence[1] + 1)/(np.sum(self.train_labels == 1) + 2)
    # evidence[2] = (evidence[2] + 1)/(np.sum(self.train_labels == 2) + 2)

    # likelihoods = np.transpose(evidence)
    # print(likelihoods)

    likelihoods = np.zeros((self.num_classes, self.num_features))

    labels = np.reshape(np.repeat(self.train_labels, 3), (900, 3))
    labels[:,0] = (labels[:,0] == 0)
    labels[:,1] = (labels[:,1] == 1)
    labels[:,2] = (labels[:,2] == 2)
    likelihoods = train_dataset.T@labels
    likelihoods = np.array(likelihoods, dtype = np.float64)

    likelihoods[:,0] = (likelihoods[:,0] + 1)/(np.sum(self.train_labels == 0) + 2)
    likelihoods[:,1] = (likelihoods[:,1] + 1)/(np.sum(self.train_labels == 1) + 2)
    likelihoods[:,2] = (likelihoods[:,2] + 1)/(np.sum(self.train_labels == 2) + 2)
    
    return likelihoods
    # return likelihoods

  def predict(self, test_dataset):
    class_prior = self.estimate_class_prior()
    yes_likelihoods = self.estimate_likelihoods()
    no_likelihoods = 1 - yes_likelihoods
    # TODO
    # return test_predict

    label0 = np.log(class_prior[0]) + np.sum(np.log(yes_likelihoods[:,0]*test_dataset + no_likelihoods[:,0]*np.invert(test_dataset)), axis = 1)
    label1 = np.log(class_prior[1]) + np.sum(np.log(yes_likelihoods[:,1]*test_dataset + no_likelihoods[:,1]*np.invert(test_dataset)), axis = 1)
    label2 = np.log(class_prior[2]) + np.sum(np.log(yes_likelihoods[:,2]*test_dataset + no_likelihoods[:,2]*np.invert(test_dataset)), axis = 1)

    test = np.array([label0, label1, label2])
    test_predict = np.argmax(test, axis = 0)

    return test_predict

if __name__ == '__main__':
  train_dataset = np.load("train_dataset.npy")
  test_dataset = np.load("test_dataset.npy")
  train_labels = np.load("train_labels.npy")
  test_labels = np.load("test_labels.npy")

  nb_model = NaiveBayesClassifier()
  nb_model.fit(train_dataset, train_labels)
  print(f"After fitting the training data, the train size is\
  {nb_model.train_size}, the number of features is {nb_model.num_features},\
  the number of class labels is {nb_model.num_classes}.") # should be 900, 2642, 3
  class_prior = nb_model.estimate_class_prior()
  print(f"The class priors are {class_prior}.") # should be [0.51495017 0.26135105 0.22369878]
  likelihoods = nb_model.estimate_likelihoods()
  print(f"The likelihoods of the first 5 features are \n {likelihoods[:5, :]}.") # should be [[0.00214592 0.00843882 0.00492611]
                                                                                 #            [0.00429185 0.00421941 0.00492611]
                                                                                 #            [0.00214592 0.00421941 0.00492611]
                                                                                 #            [0.00214592 0.00843882 0.00492611]
                                                                                 #            [0.00214592 0.00421941 0.00492611]]
  test_predict = nb_model.predict(test_dataset)
  print(f"The predictions for test data are:\n {test_predict}") # should be [0 0 1 0 0 0 1 0 0 0 0 2 0 0 1 0 0 0 0 1 1 0 0 0 0 2 0 0 0 1 2 0 0 0 0 0 0
                                                                #            0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 2 1 0 0 0 1 0 0 0 0 0 1 0 0
                                                                #            0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 2 0 0 0 2 0 0 1 0]
  accuracy_score = np.sum(test_predict == test_labels) / test_labels.shape[0]

  print(accuracy_score) # should be 0.62

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab/LAB2
After fitting the training data, the train size is  900, the number of features is 2642,  the number of class labels is 3.
The class priors are [0.51495017 0.26135105 0.22369878].
The likelihoods of the first 5 features are 
 [[0.00214592 0.00843882 0.00492611]
 [0.00429185 0.00421941 0.00492611]
 [0.00214592 0.00421941 0.00492611]
 [0.00214592 0.00843882 0.00492611]
 [0.00214592 0.00421941 0.00492611]].
The predictions for test data are:
 [0 0 1 0 0 0 1 0 0 0 0 2 0 0 1 0 0 0 0 1 1 0 0 0 0 2 0 0 0 1 2 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 2 1 0 0 0 1 0 0 0 0 0 1 0 0
 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 2 0 0 0 2 0 0 1 0]
0.62
